In [36]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
from lib.scrape import fetch_urls
from lib.write import load_jsonl_directory_pandas
from lib.extract import extract_commander_details, flatten_cards
import pandas as pd

In [38]:
commander_cards_df = pd.read_csv("../data/analytical/commander_cards.csv")

In [39]:
# commander_cards_df[(commander_cards_df["card_inclusion"] < 0.1)]
# commander_cards_df[(commander_cards_df["sanitized"].str.contains("meren-of-clan-nel-toth"))]

In [40]:
card_details_df = load_jsonl_directory_pandas("../data/operational/card_details")

In [41]:
collection_df = pd.read_csv("../data/operational/collection.csv")

In [63]:
# commander_cards = commander_cards_df[commander_cards_df["num_decks"] > 200]["card_sanitized"].unique()

# commander_cards = commander_cards_df[commander_cards_df["tribe"] == "base"]["card_sanitized"].unique()

# base_commanders_df = commander_cards_df[(commander_cards_df["tribe"] == "base") & (commander_cards_df["num_decks"] > 75)]
# commander_cards = commander_cards_df[commander_cards_df["name"].isin(base_commanders_df["name"])]["card_sanitized"].unique()

# base_commanders_df = commander_cards_df[(commander_cards_df["tribe"] == "base") & (commander_cards_df["num_decks"] > 1000)]
# commander_cards = commander_cards_df[
#     (commander_cards_df["card_inclusion"] > 0.25)
#     & commander_cards_df["name"].isin(base_commanders_df["name"])]["card_sanitized"].unique()

base_commanders_df = commander_cards_df[(commander_cards_df["tribe"] == "base") & (commander_cards_df["num_decks"] > 1000)]
commander_cards = commander_cards_df[
    (commander_cards_df["card_inclusion"] > 0.15)
    & commander_cards_df["sanitized"].isin(base_commanders_df["sanitized"])]["card_sanitized"].unique()

# TODO how many decks that have a base commander with 1000 or more decks is the card in?

# TODO does the card only show up in tribal decks?

In [64]:
unrecommended_cards_df = card_details_df[~card_details_df["sanitized"].isin(list(commander_cards))].sort_values(by="num_decks")

In [65]:
unrecommended_cards_df["sanitized"]

1011    braids-cabal-minion
410     battlefield-percher
1812       elite-javelineer
777           fleeting-aven
1001       samite-alchemist
               ...         
2014                 terror
891       shapers-sanctuary
1634     call-the-gatewatch
1936         grappling-hook
1654          mazemind-tome
Name: sanitized, Length: 788, dtype: object

In [45]:
to_remove_df = collection_df[collection_df["name_sanitized"].isin(unrecommended_cards_df["sanitized"])]

In [46]:
to_remove_df.drop(columns=["name_sanitized", "url"]).to_csv("../data/sink/nonbase.csv", index=False)